# Synaptome parameterization example

Adds physiological parameter values drawn from pathway-specific model distributions to all synapses in a synaptome (i.e., a circuit of scale "single").

In [1]:
import obi_one as obi
from entitysdk import Client, ProjectContext, models
from obi_auth import get_token
from obi_notebook import get_projects, get_entities, get_environment
from pathlib import Path

In [2]:
# environment= get_environment.get_environment()
environment = "staging"
token = get_token(environment=environment)

project_context = get_projects.get_projects(token=token, env=environment)

Dropdown(description='Select:', options=(('My first project', {'id': '7a411785-6895-4839-aaa2-d9f76e09875a', '…

In [3]:
client = Client(project_context=project_context, token_manager=token, environment=environment)

In [ ]:
synaptomes = get_entities.get_entities(
    entity_type="circuit",
    exclude_scales=["pair", "small", "microcircuit", "region", "system", "whole_brain"],
    add_columns=["build_category"],
    token=token,
    env=environment,
    project_context=project_context,
    multi_select=False,
    result=[],
)

In [26]:
# # Manual search for EM synaptome (i.e., Circuit with scale "single" and build category "em_reconstruction")
# synaptomes = client.search_entity(entity_type=models.Circuit, query={"scale": "single", "build_category": "em_reconstruction"}).all()
# assert len(synaptomes) > 0, "ERROR: No synaptomes built from EM found!"

# for syn in synaptomes:
#     print(f"{syn.name} (ID {syn.id})")

In [4]:
# Define MEModelWithSynapsesCircuitFromID by copy-pasting one of the above IDs
# synaptome_id = "<PASTE-ID-HERE>"
synaptome_id = "a1afc2bd-45a2-4e4b-9cca-bdf4e8c0d48a"
# synaptome_id = synaptomes[0]
synaptome_from_id = obi.MEModelWithSynapsesCircuitFromID(id_str=synaptome_id)

In [11]:
synapse_classes = ["excitatory_neuron", "inhibitory_neuron", "extrinsic_neuron"]
shared_within = False  # Drawn values are shared between all synapses of the same connection; otherwise values are independently drawn

# Example values for EXC synapses
distr_types_exc = {
    "conductance": {"type": "gamma", "mean": 1.0, "std": 0.5, "shared_within": shared_within},
    "syn_type_id": {"type": "constant", "mean": 100, "dtype": "int", "shared_within": shared_within},
}

# Example values for INH synapses
distr_types_inh = {
    "conductance": {"type": "gamma", "mean": 2.0, "std": 0.5, "shared_within": shared_within},
    "syn_type_id": {"type": "constant", "mean": 1, "dtype": "int", "shared_within": shared_within},
}
synapse_class_to_dict = {
    "excitatory_neuron": distr_types_exc,
    "extrinsic_neuron": distr_types_exc,
    "inhibitory_neuron": distr_types_inh
}

prop_stats = {
    _prop: {
        _src: {
            _tgt: synapse_class_to_dict[_src][_prop] for _tgt in synapse_classes
        } for _src in synapse_classes
    } for _prop in distr_types_exc.keys()
}


config_init = obi.SynapseParameterizationSingleConfig.Initialize(
    synaptome=synaptome_from_id,
    pathway_property = "synapse_class",
    pathway_param_dict={"src_types": synapse_classes,
                        "tgt_types": synapse_classes,
                        "prop_stats": prop_stats},
    overwrite_if_exists=True,
)
task_config = obi.SynapseParameterizationSingleConfig(initialize=config_init, coordinate_output_root="synaptome")

In [12]:
task = obi.SynapseParameterizationTask(config=task_config)

In [13]:
res = task.execute(db_client=client, entity_cache=True)

[2025-11-24 17:08:57,440] INFO: ConnPropsModel (w/o #syn/conn)
  Connection/synapse property distributions between 3x3 'synapse_class' pathways:
  conductance <gamma>; syn_type_id <constant (int)>
[2025-11-24 17:08:57,441] INFO: Running synapse parameterization for 1 edge population(s)...
[2025-11-24 17:08:57,940] INFO: Synapse property 'conductance' already exists in edge population 'synaptome_afferents': Re-parameterizing synapses.
[2025-11-24 17:08:57,940] INFO: Synapse property 'syn_type_id' already exists in edge population 'synaptome_afferents': Re-parameterizing synapses.
